# 밑바닥부터 시작하는 딥러닝1

***
## 신경망 학습

- 딥러닝은 end-to-end machine learning
- 모든 문제를 주어진 데이터 그대로 input으로 활용한다.
- 손실함수(loss function)를 활용해서 최적의 매개변수 값을 탐색한다.
- 일반적으로 오차제곱합과 교차 엔트로피 오차를 사용한다.
***

### 오차제곱합

<center>$E = {1 \over2}\sum_{k}(y_{k} - t_{k}) ^ 2$
    
   

In [4]:
import numpy as np

In [5]:
def sum_squared_error(y_pred, y_true) :
    
    return 0.5 * np.sum((y_pred - y_true) ** 2)

In [6]:
y_true = np.array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [7]:
y_pred = np.array([0.1, 0.05, 0.6, 0.0, 0.05, 0.1, 0.0, 0.1, 0.0, 0.0])

In [8]:
sum_squared_error(y_pred, y_true)

0.09750000000000003

***
### 교차 엔트로피 오차(cross entropy error, CEE)
<br>
<center>$E = -\sum_{k}t_{k}logy_{k}$<center>
    


사실상 target이 1일 때의 자연로그만 계산된다.

In [1]:
def cross_entropy_error(y_pred, y_true) :
    
    eps = 1e-5
    
    return -np.sum(y_true * np.log(y_pred + eps))

In [9]:
cross_entropy_error(y_pred, y_true)

0.5108089572382115

정답이 2(인덱스 기준)였고 예측 확률도 0.6으로 가장 높았기 때문에 오차가 0.51로 나타났다.

In [10]:
y_pred = np.array([0.1, 0.05, 0.1, 0.0, 0.05, 0.1, 0.0, 0.6, 0.0, 0.0])

In [11]:
cross_entropy_error(y_pred, y_true)

2.302485097993712

반면 정답 확률을 0.1로 예측하고 0.6의 확률로 예측한 클래스가 0이기 때문에 오차는 2.30으로 더 크게 나타났다.

***
### 미니배치 학습

개별 데이터에 대한 손실함수를 계산했지만 데이터의 양이 많아질수록 개별적으로 계산하는 것에 한계가 있다.

학습 데이터 중 일부만을 사용하여 근사치로 활용하는 **미니 배치**로 이러한 문제를 해결 할 수 있다.

In [24]:
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

In [14]:
data = load_digits()

In [16]:
X = data.data

In [17]:
X.shape

(1797, 64)

In [18]:
y = data.target

In [22]:
X = (X - X.mean()) / X.std()

In [25]:
tr_size = X.shape[0]

In [26]:
batch_size = 10

In [27]:
batch_mask = np.random.choice(tr_size, batch_size)

In [28]:
X_batch = X[batch_mask]

In [29]:
y_batch = y[batch_mask]

In [31]:
X_batch.shape

(10, 64)

In [32]:
y_batch

array([3, 9, 4, 3, 7, 5, 2, 1, 8, 0])

In [38]:
def cross_entropy_error(y_pred, y_true) :
    
    if y_pred.ndim == 1 :
        y_true = y_true.reshape(1, y_true.size)
        y_pred = y_true.reshape(1, y_pred.size)
        
    batch_size = y_pred.shape[0]
    
    return - np.sum(np.log(y_pred[np.arange(batch_size), y_true] + 1e-5)) / batch_size

In [40]:
y_pred

array([0.1 , 0.05, 0.1 , 0.  , 0.05, 0.1 , 0.  , 0.6 , 0.  , 0.  ])

In [41]:
y_true

array([0, 0, 1, 0, 0, 0, 0, 0, 0, 0])

In [43]:
cross_entropy_error(y_pred, y_true)

115.12925464970228

#### 왜 손실 함수를 설정하는가?

- 딥러닝의 핵심 중 하나는 "미분"으로 최적의 매개변수(가중치)를 찾기 위해서는 매개변수의 미분을 계산하고 이를 단서로 값을 갱신해 나간다.
- 즉, 매개변수 값이 조금 변화할 때 손실 함수가 어떻게 변하나? 가 핵심인 셈이다.

예를 들어 소규모 데이터에서 정확도를 지표로 삼고 신경망 학습을 진행 할 경우 정확도의 변화 자체는 미미하기 때문에 큰 변화가 없다.

하지만 손실 함수의 경우 0.1238, 0.58993 등 연속적인 값을 갖게 되기 때문에 신경망 학습에 더 적합하다고 할 수 있다.